In [1]:
import org.apache.spark.sql.{SQLContext, SaveMode}
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._

Waiting for a Spark session to start...

sqlContext = org.apache.spark.sql.SQLContext@3f85393c


org.apache.spark.sql.SQLContext@3f85393c

In [2]:
val MAG = spark.read.format("csv").option("header", "false").option("delimiter", "\t").load("file:///N/dc2/projects/IUNI_MSAcademic/mag_raw/mag/Papers.txt")
val MAG2 = MAG.select("_c0", "_c2", "_c10").toDF("MID", "Mdoi", "MJid")
MAG2.show

+----------+--------------------+----------+
|       MID|                Mdoi|      MJid|
+----------+--------------------+----------+
|2057940216|10.1111/j.1600-07...| 114728194|
|2129851794|10.1017/S00208183...| 160686149|
|1782961828|                null|  65932378|
|2361074566|                null|2764671857|
|2039151116|10.1215/00295132-...| 165333975|
|2005692652|   10.1190/1.1441619| 106174089|
|2057940218|10.1080/01416200....|  30354769|
|2129851795|10.5281/zenodo.41643|      null|
|1782961867|                null|      null|
|2361074570|                null|2764920693|
|2039151118|10.1016/j.carbon....| 188605413|
|2005692655|10.1109/ICIW.2010.45|      null|
|2057940223|10.1016/S0162-310...| 131233414|
|2129851796|10.1016/j.revpalb...|  43762691|
|1782961894|                null|      null|
|2361074586|                null|2764835777|
|2039151119|10.1016/S0005-796...| 186589312|
|2005692656|   10.1021/la027041o|  75741262|
|2057940226|10.1360/crad20070212| 166095821|
|212985179

MAG = [_c0: string, _c1: string ... 20 more fields]
MAG2 = [MID: string, Mdoi: string ... 1 more field]


[MID: string, Mdoi: string ... 1 more field]

In [2]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct

val WoS = spark.read.format("parquet").load("file:///N/dc2/projects/IUNI_MSAcademic/wos_raw/parquet/*.parquet")

val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.abstracts.abstract.abstract_text.p","static_data.summary.doctypes.doctype",
                       "static_data.summary.pub_info._pubyear","static_data.summary.names.name.display_name","static_data.summary.publishers.publisher.names.name.full_name","static_data.summary.titles.title")

val WoS3 = WoS2.withColumn("identifiers", explode(col("identifier")))
WoS3.groupBy("identifiers._type").count().show

[Stage 2:====================================================>(2219 + 1) / 2220]+---------------+--------+
|          _type|   count|
+---------------+--------+
|            doi|19854965|
|           isbn| 5185138|
|         art_no| 2759883|
|    meeting_abs| 2318856|
|          eissn|11666915|
|          eisbn| 2858055|
|   accession_no|68576611|
|           issn|63515789|
|parent_book_doi|  520045|
+---------------+--------+



WoS = [UID: string, _r_id_disclaimer: string ... 13 more fields]
WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 6 more fields]
WoS3 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 7 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 7 more fields]

In [11]:
val yearCount = WoS2.groupBy("_pubyear").count()
yearCount.show(yearCount.count.toInt, false)

[Stage 40:===================================================>(2219 + 1) / 2220]+--------+-------+
|_pubyear|count  |
+--------+-------+
|1950    |97529  |
|1936    |46915  |
|1919    |18542  |
|1951    |102476 |
|1958    |133849 |
|1921    |25381  |
|1983    |875203 |
|1905    |18486  |
|1972    |448751 |
|2007    |1992186|
|1979    |728896 |
|2014    |2760479|
|1988    |917336 |
|1908    |20847  |
|1986    |909514 |
|1949    |90748  |
|1930    |42998  |
|1969    |399411 |
|1967    |349364 |
|1964    |221303 |
|1911    |20504  |
|1956    |115670 |
|1937    |46087  |
|2012    |2552146|
|1924    |31753  |
|1991    |1046464|
|1940    |41076  |
|1918    |16499  |
|1975    |566378 |
|1945    |51526  |
|1942    |37774  |
|2016    |2919514|
|1935    |44603  |
|1902    |18075  |
|1960    |151862 |
|1934    |44400  |
|1994    |1161214|
|1923    |29597  |
|1955    |100346 |
|1987    |922854 |
|2018    |304    |
|1999    |1303803|
|1997    |1303590|
|1948    |81809  |
|1963    |196075 |
|1909   

yearCount = [_pubyear: bigint, count: bigint]


[_pubyear: bigint, count: bigint]

In [12]:
WoS.printSchema

root
 |-- UID: string (nullable = true)
 |-- _r_id_disclaimer: string (nullable = true)
 |-- abstracts: struct (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- abstract: struct (nullable = true)
 |    |    |-- abstract_text: struct (nullable = true)
 |    |    |    |-- _count: long (nullable = true)
 |    |    |    |-- p: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |-- addresses: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- address_name: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- address_spec: struct (nullable = true)
 |    |    |    |    |-- _addr_no: long (nullable = true)
 |    |    |    |    |-- city: string (nullable = true)
 |    |    |    |    |-- country: string (nullable = true)
 |    |    |    |    |-- full_address: string (nullable = true)
 |    |    |    |    |-- organizations: struct 

In [7]:
def getID(_type: String) = {
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, s, _) => s == _type}
      .map(_._3)
      .orElse(None)
  )
}


val WoS4 = WoS2.withColumn("doi", when(col("identifier").isNotNull, getID("doi")($"identifier")).otherwise(lit(null)))
.select("UID","doi","_pubyear").filter($"doi" =!= "null").coalesce(500)

WoS4.describe().filter($"summary" === "count").show

[Stage 29:=====================================================>(499 + 1) / 500]+-------+--------+--------+--------+
|summary|     UID|     doi|_pubyear|
+-------+--------+--------+--------+
|  count|19854931|19854931|19854931|
+-------+--------+--------+--------+



WoS4 = [UID: string, doi: string ... 1 more field]


getID: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[UID: string, doi: string ... 1 more field]

In [11]:
import scala.util.Try
import org.apache.hadoop.fs._
import org.apache.spark.sql.Row

def getTitle(_type: String) = { 
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, _, s) => s == _type}
      .map(_._1)
      .orElse(None)
  )
}

val WoS5 = WoS2.withColumn("journal", getTitle("source")($"title")).select("UID","doctype","journal")
.toDF("UID2","WoStype","WoSjournal")
WoS5.show

+-------------------+--------------------+--------------------+
|               UID2|             WoStype|          WoSjournal|
+-------------------+--------------------+--------------------+
|WOS:000314473000028|           [Article]|             ANALYST|
|WOS:000215261800133|           [Article]|CASE REPORTS IN M...|
|WOS:000318694100266|  [Meeting Abstract]|JOURNAL OF LABELL...|
|WOS:000318694100428|  [Meeting Abstract]|JOURNAL OF LABELL...|
|WOS:000330468400149|  [Meeting Abstract]|            PANCREAS|
|WOS:000327808700032|            [Letter]|JOURNAL OF VASCUL...|
|WOS:000322828100039|            [Letter]|APPLIED AND ENVIR...|
|WOS:000312965100005|[Editorial Material]|    GASTROENTEROLOGY|
|WOS:000332833800288|  [Meeting Abstract]|JOURNAL OF MOLECU...|
|WOS:000316290900022|[Editorial Material]|JOURNAL OF PHASE ...|
|WOS:000330358200026|           [Article]| CLASSICAL QUARTERLY|
|WOS:000331980300050|[Article, Proceed...|    BUFFALO BULLETIN|
|WOS:000331980300212|[Article, Proceed..

WoS5 = [UID2: string, WoStype: array<string> ... 1 more field]


getTitle: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[UID2: string, WoStype: array<string> ... 1 more field]

In [9]:
val MAGwos = MAG2.join(WoS4, MAG2("Mdoi")===WoS4("doi"), "inner")
MAGwos.show

[Stage 25:=====================================================>(444 + 1) / 445]+----------+--------------------+---------+-------------------+--------------------+--------+
|       MID|                Mdoi|     MJid|                UID|                 doi|_pubyear|
+----------+--------------------+---------+-------------------+--------------------+--------+
|2076045575|10.1001/2013.jama...|146641495|WOS:000312395500012|10.1001/2013.jama...|    2012|
|2074261013|10.1001/2013.jama...| 65364685|WOS:000311513000005|10.1001/2013.jama...|    2012|
|2063424611|10.1001/2013.jama...| 65364685|WOS:000312198300015|10.1001/2013.jama...|    2012|
|2139316812|10.1001/archderma...|115046667|WOS:000286229000010|10.1001/archderma...|    2011|
|2112827574|10.1001/archfacia...|193953920|WOS:000308848300010|10.1001/archfacia...|    2012|
|2099029781|10.1001/archgenps...|126359496|WOS:000299905400005|10.1001/archgenps...|    2012|
|2085073318|10.1001/archinter...| 65364685|WOS:000305594200021|10.1001/arc

MAGwos = [MID: string, Mdoi: string ... 4 more fields]


[MID: string, Mdoi: string ... 4 more fields]

In [10]:
MAGwos.write.mode(SaveMode.Overwrite).parquet("file:///N/dc2/projects/IUNI_MSAcademic/mag_raw/WoSmatch/")

[Stage 29:====================================================> (195 + 5) / 200]

In [2]:
val MAGwos2 = spark.read.format("parquet").load("file:///N/dc2/projects/IUNI_MSAcademic/mag_raw/WoSmatch/*.parquet")
MAGwos2.show

[Stage 1:=====================================================> (195 + 5) / 200]+----------+--------------------+---------+-------------------+--------------------+--------+
|       MID|                Mdoi|     MJid|                UID|                 doi|_pubyear|
+----------+--------------------+---------+-------------------+--------------------+--------+
|2165176312|10.1001/archfacia...|193953920|WOS:000306380400011|10.1001/archfacia...|    2012|
|2056420845|10.1001/archinter...| 65364685|WOS:000305087600005|10.1001/archinter...|    2012|
|2013055832|10.1001/archneuro...|164389565|WOS:000312196100001|10.1001/archneuro...|    2012|
|2015492981|10.1001/archpedia...| 14226812|WOS:000290113500010|10.1001/archpedia...|    2011|
|2110498722|10.1001/archpedia...| 14226812|WOS:000301211000003|10.1001/archpedia...|    2012|
|1999322515|10.1001/jama.2010...|172573765|WOS:000281422400025|10.1001/jama.2010...|    2010|
|2029215247|10.1001/jama.2010...|172573765|WOS:000277764500026|10.1001/jam

MAGwos2 = [MID: string, Mdoi: string ... 4 more fields]


[MID: string, Mdoi: string ... 4 more fields]

In [13]:
val MAGwos3 = MAGwos2.join(WoS5, MAGwos2("UID")===WoS5("UID2"), "leftouter")
MAGwos3.describe().filter($"summary" === "count").show

[Stage 44:=====================================================>(199 + 1) / 200]+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|summary|     MID|    Mdoi|    MJid|     UID|     doi|_pubyear|    UID2|WoSjournal|
+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|  count|18988668|18988668|17874374|18988668|18988668|18988668|18988668|  18988668|
+-------+--------+--------+--------+--------+--------+--------+--------+----------+



MAGwos3 = [MID: string, Mdoi: string ... 7 more fields]


[MID: string, Mdoi: string ... 7 more fields]

In [14]:
MAGwos3.select("MID", "Mdoi", "MJid", "UID", "_pubyear", "WoSjournal")
.toDF("MAGID", "doi", "MJid", "WoSID", "Year", "WoSjournal")
.write.mode(SaveMode.Overwrite).parquet("file:///N/dc2/projects/IUNI_MSAcademic/mag_raw/WoSmatchJournal/")

[Stage 50:=====================================================>(198 + 2) / 200]

In [5]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct

val WoS = spark.read.format("parquet").load("file:///N/dc2/projects/IUNI_MSAcademic/wos_raw/parquet/*.parquet")
val WoS5 = WoS.select("UID", "static_data.fullrecord_metadata.references.reference.uid").toDF("citing","citedList")
val WoSref2 = WoS5.join(MAGwos3, WoS5("citing")===MAGwos3("UID"), "inner")
WoSref2.describe().filter($"summary" === "count").show

Name: Unknown Error
Message: lastException: Throwable = null
<console>:56: error: not found: value MAGwos3
       val WoSref2 = WoS5.join(MAGwos3, WoS5("citing")===MAGwos3("UID"), "inner")
                               ^
<console>:56: error: not found: value MAGwos3
       val WoSref2 = WoS5.join(MAGwos3, WoS5("citing")===MAGwos3("UID"), "inner")
                                                         ^

StackTrace: 

In [21]:
val WoSref3 = WoSref2.select("citing","MID","Mdoi","_pubyear","citedList").repartition(50)

val WoSref4 = WoSref3.withColumn("cited", explode(col("citedList"))).select("citing","MID","Mdoi","_pubyear","cited")
                .toDF("citingWoS","citingMAG","citingDOI","citingYear","cited")
//WoSref4.describe().filter($"summary" === "count").show

WoSref3 = [citing: string, MID: string ... 3 more fields]
WoSref4 = [citingWoS: string, citingMAG: string ... 3 more fields]


lastException: Throwable = null


[citingWoS: string, citingMAG: string ... 3 more fields]

In [ ]:
val MAGwos4 = MAGwos3.select("MID", "Mdoi","UID", "_pubyear").toDF("citedMAG", "citedDOI", "citedWoS", "Year")
val WoSref5 = WoSref4.join(broadcast(MAGwos4), WoSref4("cited")===MAGwos4("citedWoS"), "inner")
WoSref5.describe().filter($"summary" === "count").show

[Stage 73:=========================================>           (157 + 43) / 200]

In [ ]:
WoSref4.write.mode(SaveMode.Overwrite).parquet("file:///N/dc2/projects/IUNI_MSAcademic/mag_raw/WoStallCitation/")